In [1]:
import pandas as pd

In [5]:
data = pd.read_csv("data_test1.csv", names=["date", "cost", "description", "category"])


In [10]:
print(data)

           date     cost                               description  \
0    2014-08-11   184.22                               Ski tickets   
1    2011-09-21  4034.86                               Edfinancial   
2    2010-05-27   283.24                                  GME YOLO   
3    2019-02-21  9012.58                                  Pet food   
4    2015-01-09   850.34                                  GME YOLO   
..          ...      ...                                       ...   
245  2012-01-20  5812.81    Deposit in savings at Bank of the West   
246  2018-01-10  3449.14  Blue Cross Bleu Shield of North Carolina   
247  2017-07-22  5059.13                               Capital One   
248  2010-05-01  1023.06                                ExxonMobil   
249  2015-10-17   233.35                                  Vacation   

               category  
0        Non-Essentials  
1             Education  
2                 Other  
3             Education  
4                 Other  
.. 